In [1]:
import os
import pickle

import pandas as pd

from graphs import load_dataset_to_graph
from measurements import get_graph_measurements, compare_graph_measurements, \
    print_comparison_results, collect_graph_measurements, MEASUREMENTS
from recreate_graph import recreate_by_priority_rank_random_rankings, \
    recreate_by_priority_rank, get_trained_model

Using TensorFlow backend.
/home/smoq/.virtualenvs/cna/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
prepared_datasets_path = 'prepared_datasets'
prepared_dataframes_path = 'prepared_dataframes'
delimiter = '\t'

prepared_dataset_names = [
    #'primary_school', 'workplace', 
    'highschool_2011', #'highschool_2012', 'hospital', 
    #'moreno_blogs', 'moreno_sheep', 'moreno_seventh',
    # big datasets
    #'petster-hamster', 'email-Eu'
]

comparison_dir = 'results'
number_of_comparisons = 10

In [3]:
for dataset_name in prepared_dataset_names:
    # paths
    dataset_path = os.path.join(prepared_datasets_path, dataset_name)
    dataframe_path = os.path.join(prepared_dataframes_path, dataset_name)
    results_dir = os.path.join(comparison_dir, dataset_name)
    original_file = os.path.join(results_dir, 'original.csv')
    
    if not os.path.exists(results_dir):
        os.makedirs(results_dir)
         
    # load original graph and get measurements
    graph = load_dataset_to_graph(dataset_path)
    graph_measurements = get_graph_measurements(graph)
    
    # make 5 types of analysis
    for analysis_name in ['random_rankings', 'graph_attrs', 'real_attrs', 'graph_real_attrs', 'real_y']:
        print('{}'.format(analysis_name))
        results_file = os.path.join(results_dir, analysis_name + '.csv')
        results = []
        
        # handle random rankings case
        if analysis_name == 'random_rankings':
            for i in range(number_of_comparisons):
                print('Recreation number {}...'.format(i))
                new_graph = recreate_by_priority_rank_random_rankings(graph)

                new_graph_measurements = get_graph_measurements(new_graph)
                measurement_values = collect_graph_measurements(graph_measurements, new_graph_measurements)
                results.append(measurement_values)
        # handle real y case
        elif analysis_name == 'real_y':
            df_path = os.path.join(dataframe_path, 'no_attrs.csv')
            df = pd.read_csv(df_path, delimiter=delimiter)
            
            for i in range(number_of_comparisons):
                print('Recreation number {}...'.format(i))
                new_graph = recreate_by_priority_rank(graph, df['num_of_edges'].as_matrix())
                new_graph_measurements = get_graph_measurements(new_graph)
                
                measurement_values = collect_graph_measurements(graph_measurements, new_graph_measurements)
                results.append(measurement_values)
        # handle graph attrs, real attrs, graph and real attrs cases
        else:
            df_path = os.path.join(dataframe_path, analysis_name + '.csv')
            df = pd.read_csv(df_path, delimiter=delimiter)
            
            model = get_trained_model(df, epochs=16)
            # drop target column
            X_test = df.drop(['num_of_edges'], axis=1)
            # predict num_edges
            y_pred = model.predict(X_test)
            
            for i in range(number_of_comparisons):
                print('Recreation number {}...'.format(i))
                new_graph = recreate_by_priority_rank(graph, y_pred)
                new_graph_measurements = get_graph_measurements(new_graph)
                
                measurement_values = collect_graph_measurements(graph_measurements, new_graph_measurements)
                results.append(measurement_values)
            
        results_df = pd.DataFrame(results)
        results_df.to_csv(results_file, index=False, sep=delimiter)
        
    scalar_measurements = {k: v for k, v in graph_measurements.items() if MEASUREMENTS[k] == 'value'}
    orig_df = pd.DataFrame([scalar_measurements])
    orig_df.to_csv(original_file, index=False, sep=delimiter)

random_rankings
Recreation number 0...


/home/smoq/.virtualenvs/cna/lib/python3.6/site-packages/networkx/algorithms/assortativity/correlation.py:287: RuntimeWarning: divide by zero encountered in double_scalars
  return (xy*(M-ab)).sum()/numpy.sqrt(vara*varb)


Recreation number 1...
Recreation number 2...
Recreation number 3...
Recreation number 4...
Recreation number 5...
Recreation number 6...
Recreation number 7...
Recreation number 8...
Recreation number 9...
graph_attrs
Epoch 1/16
15876/15876 [==============================] - 0s 20us/step - loss: 0.2198 - acc: 0.7590
Epoch 2/16
15876/15876 [==============================] - 0s 15us/step - loss: 0.1688 - acc: 0.7846
Epoch 3/16
15876/15876 [==============================] - 0s 15us/step - loss: 0.1299 - acc: 0.7846
Epoch 4/16
15876/15876 [==============================] - 0s 14us/step - loss: 0.1005 - acc: 0.7846
Epoch 5/16
15876/15876 [==============================] - 0s 14us/step - loss: 0.0784 - acc: 0.7846
Epoch 6/16
15876/15876 [==============================] - 0s 16us/step - loss: 0.0619 - acc: 0.7846
Epoch 7/16
15876/15876 [==============================] - 0s 15us/step - loss: 0.0495 - acc: 0.7846
Epoch 8/16
15876/15876 [==============================] - 0s 15us/step - loss: 0.

/home/smoq/.virtualenvs/cna/lib/python3.6/site-packages/networkx/algorithms/assortativity/correlation.py:287: RuntimeWarning: invalid value encountered in double_scalars
  return (xy*(M-ab)).sum()/numpy.sqrt(vara*varb)


Recreation number 2...
Recreation number 3...
Recreation number 4...
Recreation number 5...
Recreation number 6...
Recreation number 7...
Recreation number 8...
Recreation number 9...
real_attrs
Epoch 1/16
15876/15876 [==============================] - 0s 17us/step - loss: 0.0849 - acc: 0.7320
Epoch 2/16
15876/15876 [==============================] - 0s 15us/step - loss: 0.0013 - acc: 0.7846
Epoch 3/16
15876/15876 [==============================] - 0s 15us/step - loss: 8.6006e-04 - acc: 0.7846
Epoch 4/16
15876/15876 [==============================] - 0s 15us/step - loss: 7.9431e-04 - acc: 0.7846
Epoch 5/16
15876/15876 [==============================] - 0s 16us/step - loss: 7.7333e-04 - acc: 0.7846
Epoch 6/16
15876/15876 [==============================] - 0s 15us/step - loss: 7.6472e-04 - acc: 0.7846
Epoch 7/16
15876/15876 [==============================] - 0s 14us/step - loss: 7.6064e-04 - acc: 0.7846
Epoch 8/16
15876/15876 [==============================] - 0s 16us/step - loss: 7.5846

/home/smoq/.virtualenvs/cna/lib/python3.6/site-packages/networkx/algorithms/assortativity/correlation.py:287: RuntimeWarning: invalid value encountered in sqrt
  return (xy*(M-ab)).sum()/numpy.sqrt(vara*varb)
